This notebook processes and analyzes data related to TEM1 combinatorial mutagenesis under ampicillin and aztreonam selection. It reads raw data, generates genotype strings, filters data based on intended mutations, converts data to long format, calculates statistics (mean, median, std, cv_std), and identifies missing mutants. Finally, it saves the processed dataframes to parquet files.

In [1]:
import polars as pl
import numpy as np
import pandas as pd
from itertools import product
from scipy import stats
from collections import defaultdict
from pathlib import Path

In [2]:
PROJECT_PATH = Path("/project/greencenter/Toprak_lab/shared/TEM1_Combinatorial_Mutagenesis/src/Epistasis")
amp_path = PROJECT_PATH / "data/raw/Ampicillin_auc_per_genotype.csv"
azt_path = PROJECT_PATH / "data/raw/Aztreonam_auc_per_genotype.csv"

# Read the data into Polars DataFrames
amp_df = pl.read_csv(amp_path)
azt_df = pl.read_csv(azt_path)

# Define the intended mutations at specific positions
intended = {19: ['.','P'], 37: ['.','K'], 67: ['.','L','V'],
            102: ['.','K'], 162: ['.','S','H','N'],
            180: ['.','T'], 235: ['.','T'], 236: ['.','S'],
            237: ['.','K'], 241: ['.','S','C'], 261: ['.','M'],
            271: ['.','L','Q'], 272:['.','D']}

# Extract the mutant profiles from the Ampicillin DataFrame
mutants_id_arr = amp_df["mut_profile_masked"].to_numpy()
mutants_id_arr = mutants_id_arr.astype(str)

# Generate all possible combinations of the intended mutations
mutation_combinations = product(*[intended[pos] for pos in sorted(intended.keys())])

# Convert the mutation combinations to strings
mutation_combinations = ["".join(mut) for mut in mutation_combinations]
possible_intended_mutants = np.array(mutation_combinations)

# Define the wildtype and dead mutant sequences
wildtype = "............."
dead_mutant = "XXXXXXXXXXXXX"
possible_intended_mutants = np.append(possible_intended_mutants, [wildtype, dead_mutant], axis=0)

# Filter the DataFrames to keep only rows with the intended mutations
amp_df = amp_df.filter(amp_df["mut_profile_masked"].is_in(possible_intended_mutants))
azt_df = azt_df.filter(azt_df["mut_profile_masked"].is_in(possible_intended_mutants))

# Function to clean rows with too many null values (missing replicates)
def clean_nulls(df):
    # Count the number of null values in each row
    df_with_null_counts = df.with_columns(
        pl.sum_horizontal(pl.all().is_null()).alias('null_count_per_row')
    )
    # Define a threshold for the maximum number of allowed nulls (more than 2 replicates missing)
    threshold = (df.shape[1] - 3) *2 /3 # delete rows with more than 2 replicates missing
    # Filter the DataFrame to keep only rows with null counts below the threshold
    filtered_df = df_with_null_counts.filter(pl.col('null_count_per_row') < threshold)

    # Remove the null count column
    filtered_df = filtered_df.drop('null_count_per_row')
    return filtered_df

# Clean the Ampicillin and Aztreonam DataFrames
amp_df = clean_nulls(amp_df)
azt_df = clean_nulls(azt_df)

In [3]:
amp_df

,mut_profile_masked,Ampicillin 0.0 1,Ampicillin 0.0 2,Ampicillin 0.0 3,Ampicillin 3.1 1,Ampicillin 3.1 2,Ampicillin 3.1 3,Ampicillin 12.2 1,Ampicillin 12.2 2,Ampicillin 12.2 3,Ampicillin 48.8 1,Ampicillin 48.8 2,Ampicillin 48.8 3,Ampicillin 195.0 1,Ampicillin 195.0 2,Ampicillin 195.0 3,Ampicillin 781.0 1,Ampicillin 781.0 2,Ampicillin 781.0 3
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
4191,"""..VK.T..K...D""",3.246056,3.381039,3.386168,3.254894,3.499717,3.572811,3.445461,3.555552,3.707558,3.688961,3.724072,3.717756,3.926324,4.054691,3.966381,4.318668,4.576903,4.574156
15454,"""PK.K....K....""",3.297042,3.029911,3.374664,3.367117,3.155275,3.360832,3.556505,3.286163,3.568653,3.719924,3.38432,3.656833,3.866684,3.751909,3.992477,4.574223,4.295283,4.534542
44010,""".K.KNT....M.D""",3.19552,3.521552,3.360132,3.250648,3.487157,3.404783,3.350543,3.613922,3.519203,3.571188,3.787627,3.574253,3.848914,4.089437,3.905243,4.270981,4.564519,4.552179
51705,"""....H.....MLD""",3.306043,3.068982,3.275571,3.350747,3.202053,3.492762,3.592171,3.280893,3.497878,3.727503,3.520945,3.733589,4.061591,3.785649,4.036152,4.474429,4.32325,4.554319
30703,""".K.KST..K..L.""",3.320329,3.264925,3.153442,3.517775,3.396093,3.293433,3.509525,3.646681,3.495351,3.569673,3.777665,3.687904,4.000945,3.979897,3.82414,4.505631,4.562776,4.422571
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
58437,"""PK...TT.KSMQ.""",2.93213,3.187853,null,3.051794,2.857095,null,1.346469,2.917537,null,2.337924,1.590514,null,0.176091,2.712654,null,0.176091,1.409214,null
56999,"""PKVKSTTSKCMLD""",2.884183,3.035578,2.986852,3.004383,2.854704,2.986936,2.412882,2.690296,2.867564,2.946985,1.650166,1.467529,2.29075,0.176091,0.176091,0.624801,0.176091,0.176091
44329,"""PKLKNTTSKCMQ.""",3.106641,null,2.865794,2.829679,null,2.768964,2.147385,null,2.49301,2.543131,null,2.060003,2.410405,null,0.176091,0.805302,null,0.926563


In [4]:
def convert_long_format(df, concs):
    """
    Converts a DataFrame from a wide format (where each concentration and replicate is a column)
    to a long format (where each row represents a single measurement at a specific concentration
    and replicate).

    Args:
        df (pl.DataFrame): The input DataFrame in wide format.  It is expected to have a column
                           named 'mut_profile_masked' containing the mutant profiles, and several
                           columns named according to the concentrations and replicates (e.g.,
                           "Ampicillin 0.0 1", "Ampicillin 0.0 2", etc.).
        concs (list): A list of strings representing the concentrations used in the experiment
                      (e.g., ["Ampicillin 0.0", "Ampicillin 3.1", ...]).

    Returns:
        pl.DataFrame: A DataFrame in long format with columns 'mutant_profile', 'concentration',
                      'replicate1', 'replicate2', and 'replicate3'.
    """

    # Create a list of column names for all replicates at all concentrations.
    # For example, if concs = ["Ampicillin 0.0", "Ampicillin 3.1"] and there are 3 replicates,
    # rep_concs will be ["Ampicillin 0.0 1", "Ampicillin 0.0 2", "Ampicillin 0.0 3",
    #                   "Ampicillin 3.1 1", "Ampicillin 3.1 2", "Ampicillin 3.1 3"].
    rep_concs = [f"{conc} {i+1}" for conc in concs for i in range(3)]

    # Extract the data from the specified columns into a NumPy array.
    data_array = df[rep_concs].to_numpy()

    # Get the mutant profiles from the 'mut_profile_masked' column.
    mutant_profiles = df['mut_profile_masked'].to_list()

    # Get the dimensions of the data array.
    n_mutants, n_columns = data_array.shape

    # Calculate the number of concentrations and replicates.
    n_concs = len(concs)
    n_replicates = n_columns // n_concs

    # Reshape the data array to group the replicates for each mutant and concentration.
    # The reshaped array will have dimensions (n_mutants, n_concs, n_replicates).
    reshaped_data = data_array.reshape(n_mutants, n_concs, n_replicates)

    # Initialize lists to store the data in long format.
    long_mutants = []
    long_concs = []
    long_rep1 = []
    long_rep2 = []
    long_rep3 = []

    # Iterate over the mutant profiles and concentrations to populate the long format lists.
    for i, mutant in enumerate(mutant_profiles):
        for j, conc in enumerate(concs):
            long_mutants.append(mutant)

            # Extract only the concentration value (e.g., "0.0" from "Ampicillin 0.0").
            conc = conc.split(" ")[1]
            long_concs.append(float(conc))

            # Append the replicate values to the corresponding lists. If there are fewer than 3
            # replicates, append None to the missing replicate lists.
            long_rep1.append(reshaped_data[i, j, 0])
            long_rep2.append(reshaped_data[i, j, 1] if n_replicates > 1 else None)
            long_rep3.append(reshaped_data[i, j, 2] if n_replicates > 2 else None)

    # Create a Polars DataFrame from the long format lists.
    long_df = pl.DataFrame({
        'mutant_profile': long_mutants,
        'concentration': long_concs,
        'replicate1': long_rep1,
        'replicate2': long_rep2,
        'replicate3': long_rep3
    })

    return long_df

# Define the concentrations for Ampicillin and Aztreonam.
amp_concs = ["Ampicillin 0.0", "Ampicillin 3.1", "Ampicillin 12.2", "Ampicillin 48.8", "Ampicillin 195.0", "Ampicillin 781.0"]
azt_concs = ["Aztreonam 0.0", "Aztreonam 0.44", "Aztreonam 1.33", "Aztreonam 4.0", "Aztreonam 12.0", "Aztreonam 36.0", "Aztreonam 108.0", "Aztreonam 324.0"]

# Convert the Ampicillin and Aztreonam DataFrames to long format.
amp_long_df = convert_long_format(amp_df, amp_concs)
azt_long_df = convert_long_format(azt_df, azt_concs)

Calculate_statistics function below had a problem. It was calculating fitness by converting fitness values to 10^ fitness and doing statistics accordingly. It was then calculating mean, stdev etc. It is now fixed. We calculate fitness for each replicate, keep it as is, and do statistics accordingly.

In [5]:
def calculate_statistics(df):
    """
    Calculates statistics (mean, median, standard deviation, and coefficient of variation)
    for each mutant profile at each concentration, based on the replicate data.

    Args:
        df (pl.DataFrame): A Polars DataFrame in long format, containing columns for
                           'mutant_profile', 'concentration', 'replicate1', 'replicate2',
                           and 'replicate3'.

    Returns:
        pl.DataFrame: A Polars DataFrame with the same columns as the input, plus additional
                           columns for 'mean', 'median', 'std', and 'cv_std', representing
                           the calculated statistics.
    """
    # Extract the replicate data into NumPy arrays
    rep1 = df['replicate1'].to_numpy()
    rep2 = df['replicate2'].to_numpy()
    rep3 = df['replicate3'].to_numpy()

    # Stack the replicate arrays into a single array for easier calculations
    rep_arr = np.stack([rep1, rep2, rep3], axis=1)

    # Calculate the mean across replicates, handling NaN values
    mean = np.nanmean(rep_arr, axis=1)

    # Calculate the median across replicates, handling NaN values
    median = np.nanmedian(rep_arr, axis=1)

    # Calculate the standard deviation across replicates, handling NaN values
    std = np.nanstd(rep_arr, axis=1)

    # Calculate the coefficient of variation (CV = std / mean), handling potential division by zero
    cv_std = std / mean

    # Create a new Polars DataFrame to store the results
    stats_df = pl.DataFrame({
        'mutant_profile': df['mutant_profile'],  # Copy mutant profiles
        'concentration': df['concentration'],  # Copy concentrations
        'replicate1': df['replicate1'],  # Copy replicate 1 data
        'replicate2': df['replicate2'],  # Copy replicate 2 data
        'replicate3': df['replicate3'],  # Copy replicate 3 data
        'mean': mean,  # Store the calculated mean
        'median': median,  # Store the calculated median
        'std': std,  # Store the calculated standard deviation
        'cv_std': cv_std,  # Store the calculated coefficient of variation
    })

    return stats_df  # Return the DataFrame with statistics

In [6]:
amp_stats_df = calculate_statistics(amp_long_df)
azt_stats_df = calculate_statistics(azt_long_df)

In [7]:
amp_stats_df = amp_stats_df.sort("mutant_profile")
azt_stats_df = azt_stats_df.sort("mutant_profile")

In [8]:
amp_stats_df

mutant_profile,concentration,replicate1,replicate2,replicate3,mean,median,std,cv_std
str,f64,f64,f64,f64,f64,f64,f64,f64
""".............""",0.0,3.305258,3.148729,3.409926,3.287971,3.305258,0.107332,0.032644
""".............""",3.1,3.336995,3.149033,3.302661,3.262896,3.302661,0.081724,0.025047
""".............""",12.2,3.332327,3.235903,3.540045,3.369425,3.332327,0.126906,0.037664
""".............""",48.8,3.641417,3.500858,3.740226,3.6275,3.641417,0.098216,0.027075
""".............""",195.0,3.843543,3.667657,3.986915,3.832705,3.843543,0.130562,0.034065
…,…,…,…,…,…,…,…,…
"""XXXXXXXXXXXXX""",3.1,2.283756,2.124584,2.440196,2.282845,2.283756,0.12885,0.056443
"""XXXXXXXXXXXXX""",12.2,1.704191,1.755785,1.767616,1.742531,1.755785,0.027537,0.015803
"""XXXXXXXXXXXXX""",48.8,0.864645,0.839891,0.845641,0.850059,0.845641,0.010578,0.012443


In [9]:
azt_stats_df

mutant_profile,concentration,replicate1,replicate2,replicate3,mean,median,std,cv_std
str,f64,f64,f64,f64,f64,f64,f64,f64
""".............""",0.0,3.145844,3.206299,3.25207,3.201404,3.206299,0.043505,0.013589
""".............""",0.44,2.787756,2.966409,3.007675,2.920613,2.966409,0.095443,0.032679
""".............""",1.33,2.449747,2.790716,2.86212,2.700861,2.790716,0.179941,0.066624
""".............""",4.0,2.283766,2.433078,2.663517,2.46012,2.433078,0.156208,0.063496
""".............""",12.0,2.393637,2.374871,2.563496,2.444001,2.393637,0.084842,0.034714
…,…,…,…,…,…,…,…,…
"""XXXXXXXXXXXXX""",4.0,2.103255,2.002753,2.065443,2.05715,2.065443,0.041447,0.020148
"""XXXXXXXXXXXXX""",12.0,2.113278,2.041799,2.058811,2.071296,2.058811,0.030487,0.014719
"""XXXXXXXXXXXXX""",36.0,2.105288,2.009686,1.99986,2.038278,2.009686,0.047553,0.02333


In [10]:
# select the wildtype and dead mutant
amp_long_df.filter(pl.col("mutant_profile").is_in([wildtype, dead_mutant]))

mutant_profile,concentration,replicate1,replicate2,replicate3
str,f64,f64,f64,f64
""".............""",0.0,3.305258,3.148729,3.409926
""".............""",3.1,3.336995,3.149033,3.302661
""".............""",12.2,3.332327,3.235903,3.540045
""".............""",48.8,3.641417,3.500858,3.740226
""".............""",195.0,3.843543,3.667657,3.986915
…,…,…,…,…
"""XXXXXXXXXXXXX""",3.1,2.283756,2.124584,2.440196
"""XXXXXXXXXXXXX""",12.2,1.704191,1.755785,1.767616
"""XXXXXXXXXXXXX""",48.8,0.864645,0.839891,0.845641


In [11]:
azt_stats_df.filter(pl.col("mutant_profile").is_in([wildtype, dead_mutant]))

mutant_profile,concentration,replicate1,replicate2,replicate3,mean,median,std,cv_std
str,f64,f64,f64,f64,f64,f64,f64,f64
""".............""",0.0,3.145844,3.206299,3.25207,3.201404,3.206299,0.043505,0.013589
""".............""",0.44,2.787756,2.966409,3.007675,2.920613,2.966409,0.095443,0.032679
""".............""",1.33,2.449747,2.790716,2.86212,2.700861,2.790716,0.179941,0.066624
""".............""",4.0,2.283766,2.433078,2.663517,2.46012,2.433078,0.156208,0.063496
""".............""",12.0,2.393637,2.374871,2.563496,2.444001,2.393637,0.084842,0.034714
…,…,…,…,…,…,…,…,…
"""XXXXXXXXXXXXX""",4.0,2.103255,2.002753,2.065443,2.05715,2.065443,0.041447,0.020148
"""XXXXXXXXXXXXX""",12.0,2.113278,2.041799,2.058811,2.071296,2.058811,0.030487,0.014719
"""XXXXXXXXXXXXX""",36.0,2.105288,2.009686,1.99986,2.038278,2.009686,0.047553,0.02333


In [12]:
# Define the positions of interest in the TEM1 sequence (Ambler numbering)
sign_pos = [19,  37,  67, 102, 162, 180, 235, 236, 237, 241, 261, 271, 272] #Ambler numbering is different than residue numbering
sign_pos_Ambler = [21,  39,  69, 104, 164, 182, 237, 238, 240, 244, 265, 275, 276]

# Define the intended mutations at each position of interest
# Each position is a key in the dictionary, and the value is a list of amino acids to which the position is intended to be mutated
intended = {19: ['L','P'], 37: ['Q','K'], 67: ['M','L','V'],
            102: ['E','K'], 162: ['R','S','H','N'],
            180: ['M','T'], 235: ['A','T'], 236: ['G','S'],
            237: ['E','K'], 241: ['R','S','C'], 261: ['T','M'],
            271: ['R','L','Q'], 272:['N','D']}

# Define all possible mutations at each position (including the wild-type)
# This dictionary includes the wild-type amino acid (".") in addition to the intended mutations
all_mutations = {19: ['.','P'], 37: ['.','K'], 67: ['.','L','V'],
        102: ['.','K'], 162: ['.','S','H','N'],
        180: ['.','T'], 235: ['.','T'], 236: ['.','S'],
        237: ['.','K'], 241: ['.','S','C'], 261: ['.','M'], 271: ['.','L','Q'], 272:['.','D']}

In [13]:
# Calculate the number of mutation options at each position
def mutation_options(intended):
    return [len(aas) for pos, aas in intended.items()]


# Calculate the total number of mutants
intended_mutants_size = np.prod(mutation_options(intended))

# Generate a list of all possible mutants as binary strings
intended_mutants_all = [''.join(mut) for mut in product(*[intended[pos] for pos in sorted(intended.keys())])]
all_mutants = [''.join(mut) for mut in product(*[all_mutations[pos] for pos in sorted(all_mutations.keys())])]

# Combine the two lists into a DataFrame
intended_mutants_df = pd.DataFrame({'intended_mutants': intended_mutants_all, 'all_mutants': all_mutants})

# Generate a matching binary string of 13 characters
def generate_binary_string(mutant, intended):
    return ''.join(str(intended[pos].index(mutant[i])) if mutant[i] != "." else "0" for i, pos in enumerate(sorted(intended.keys())))

intended_mutants_df['binary'] = intended_mutants_df['intended_mutants'].apply(lambda x: generate_binary_string(x, intended))

# Add a column for the number of mutations in each mutant
intended_mutants_df['num_mutations'] = intended_mutants_df['binary'].apply(lambda x: sum(1 for char in x if char != '0'))

# Sort the DataFrame by the number of mutations and binary string
intended_mutants_df = intended_mutants_df.sort_values(['binary']).reset_index(drop=True)

# Add a column for the Hamming distance between the intended_mutants and all_mutants, this measures the number of positions that differ between the two strings: intended and all mutants
intended_mutants_df['hamming'] = intended_mutants_df.apply(lambda x: sum(1 for i in range(len(x['intended_mutants'])) if x['intended_mutants'][i] != x['all_mutants'][i]), axis=1)

#INTENDED MUTANTS ARE GENERATED BY MUTATING THE WILDTYPE SEQUENCE WITH THE INTENDED MUTATIONS AT POSITIONS 19, 37, 67, 102, 162, 180, 235, 236, 237, 241, 261, 271, 272
# POSITION 19 IS THE FIRST CHARACTER OF THE BINARY STRING, POSITION 37 IS THE SECOND CHARACTER OF THE BINARY STRING, AND SO ON
intended_mutants_df

,intended_mutants,all_mutants,binary,num_mutations,hamming
0,LQMERMAGERTRN,.............,0000000000000,0,13
1,LQMERMAGERTRD,............D,0000000000001,1,12
2,LQMERMAGERTLN,...........L.,0000000000010,1,12
3,LQMERMAGERTLD,...........LD,0000000000011,2,11
4,LQMERMAGERTQN,...........Q.,0000000000020,1,12
...,...,...,...,...,...
55291,PKVKNTTSKCMRD,PKVKNTTSKCM.D,1121311112101,12,1
55292,PKVKNTTSKCMLN,PKVKNTTSKCML.,1121311112110,12,1
55293,PKVKNTTSKCMLD,PKVKNTTSKCMLD,1121311112111,13,0
55294,PKVKNTTSKCMQN,PKVKNTTSKCMQ.,1121311112120,12,1


In [14]:
#missing mutants in the dataset
intended_mutants_df[~intended_mutants_df['all_mutants'].isin(azt_df['mut_profile_masked'])]

,intended_mutants,all_mutants,binary,num_mutations,hamming
46390,PKLERTAGESMQN,PKL..T...SMQ.,1110010001120,7,6
51062,PKVERTASESTLN,PKV..T.S.S.L.,1120010101010,7,6
53444,PKVKRTTGESMLN,PKVK.TT..SML.,1121011001110,9,4


In [15]:
#missing mutants in the dataset
intended_mutants_df[~intended_mutants_df['all_mutants'].isin(amp_df['mut_profile_masked'])]

,intended_mutants,all_mutants,binary,num_mutations,hamming
47155,PKLESTTGKCMRD,PKL.STT.KCM.D,1110111012101,10,3
51154,PKVERTTGECMQN,PKV..TT..CMQ.,1120011002120,8,5
54011,PKVKSTTGERMQD,PKVKSTT...MQD,1121111000121,10,3


In [ ]:
amp_df.write_parquet(PROJECT_PATH / "data/processed/amp_auc_wide_df.parquet")
azt_df.write_parquet(PROJECT_PATH / "data/processed/azt_auc_wide_df.parquet")
# amp_long_df.write_parquet(PROJECT_PATH / "amp_auc_df_long.parquet")
# azt_long_df.write_parquet(PROJECT_PATH / "azt_auc_df_long.parquet")
amp_stats_df.write_parquet(PROJECT_PATH / "data/processed/amp_auc_long_df.parquet")
azt_stats_df.write_parquet(PROJECT_PATH / "data/processed/azt_auc_long_df.parquet")